- 시계열 데이터에 대한 예측
    - sunspots dataset : 단변량 시계열 데이터
    - HEPC (houshold electric power consumption) dataset : 다변량 시계열 데이터
- tensorflow로 모델 개발
- torch로 동일한 모델 개발

# Sunspots dataset

In [187]:
import numpy as np
import pandas as pd
import urllib

url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
urllib.request.urlretrieve(url, 'sunspots.csv')
sunspots_df = pd.read_csv('sunspots.csv')
sunspots_df.head()

,Unnamed: 0,Date,Monthly Mean Total Sunspot Number
0,0,1749-01-31,96.7
1,1,1749-02-28,104.3
2,2,1749-03-31,116.7
3,3,1749-04-30,92.8
4,4,1749-05-31,141.7


In [188]:
time_step = sunspots_df['Date'].values
sunspots = sunspots_df['Monthly Mean Total Sunspot Number'].values
sunspots_max = sunspots.max()
sunspots_min = sunspots.min()
sunspots = (sunspots - sunspots_min) / (sunspots_max - sunspots_min)

In [189]:
split_time = 3000
X_train  = sunspots[:split_time]
X_valid = sunspots[split_time:]
time_train = time_step[:split_time]
time_valid = time_step[split_time:]

# tensorflow

In [190]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    # 종속변수와 독립변수를 나눔
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [191]:
import tensorflow as tf
from tensorflow import keras

# 0~29의 데이터를 이용해 뒤의 1~30의 데이터를 예측함
window_size = 30
batch_size = 32
shuffle_buffer = 1000

tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

train_set = windowed_dataset(X_train, window_size, batch_size, shuffle_buffer)
valid_set = windowed_dataset(X_valid, window_size, batch_size, shuffle_buffer)

In [192]:
for i,(x,y) in enumerate(train_set.take(1)):
    if i==1:
        break

In [193]:
x.shape, y.shape

(TensorShape([32, 30, 1]), TensorShape([32, 30, 1]))

In [194]:
# first batch
x[0,:,0]

<tf.Tensor: shape=(30,), dtype=float64, numpy=
array([0.185334  , 0.18257157, 0.16248117, 0.09718734, 0.20015068,
       0.23606228, 0.1594676 , 0.12506278, 0.15243596, 0.24234053,
       0.40256153, 0.11049724, 0.08864892, 0.16750377, 0.20919136,
       0.18834756, 0.15369161, 0.10723255, 0.1210447 , 0.11878453,
       0.14590658, 0.09392265, 0.10647916, 0.1443998 , 0.22225013,
       0.15268709, 0.11752888, 0.13209442, 0.10899046, 0.13259669])>

In [195]:
np.where(sunspots == x[0,0,0].numpy())

(array([184, 652, 808]),)

In [199]:
sunspots[808:808+window_size]

array([0.185334  , 0.18257157, 0.16248117, 0.09718734, 0.20015068,
       0.23606228, 0.1594676 , 0.12506278, 0.15243596, 0.24234053,
       0.40256153, 0.11049724, 0.08864892, 0.16750377, 0.20919136,
       0.18834756, 0.15369161, 0.10723255, 0.1210447 , 0.11878453,
       0.14590658, 0.09392265, 0.10647916, 0.1443998 , 0.22225013,
       0.15268709, 0.11752888, 0.13209442, 0.10899046, 0.13259669])

In [200]:
# 0~29의 데이터를 이용해 뒤의 1~30의 데이터를 예측함
a,_ = tf.unique(tf.reshape(x[:,1:,:] == y[:,:-1,:],-1))
a

<tf.Tensor: shape=(1,), dtype=bool, numpy=array([ True])>

In [201]:
# 참고
print(x.shape)
x_conv = keras.layers.Conv1D(filters=60, kernel_size=5, padding='causal', activation='relu', input_shape=[None, 1])(x)
print(x_conv.shape)
x_lstm1 = keras.layers.LSTM(60, return_sequences=True)(x_conv)
print(x_lstm1.shape)
x_lstm2 = keras.layers.LSTM(60, return_sequences=True)(x_lstm1)
print(x_lstm2.shape)
x_dense1 = keras.layers.Dense(30, activation='relu')(x_lstm2)
print(x_dense1.shape)

(32, 30, 1)
(32, 30, 60)
(32, 30, 60)
(32, 30, 60)
(32, 30, 30)


In [202]:
model = keras.models.Sequential([
    # causal padding은 시계열 데이터에서 미래의 데이터를 미리 보는 일을 방지하기 위해 과거 데이터쪽으로만 padding을 추가
    keras.layers.Conv1D(filters=60, kernel_size=5, padding='causal', activation='relu', input_shape=[None, 1]),
    keras.layers.LSTM(60, return_sequences=True),
    keras.layers.LSTM(60, return_sequences=True),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1)
])

optimizer = keras.optimizers.SGD(lr=1e-5, momentum=0.9)
loss = keras.losses.Huber()
model.compile(loss=loss, optimizer=optimizer, metrics=['mae'])

# callback은 생략
model.fit(train_set, epochs=40, validation_data=valid_set)

Epoch 1/40
93/93 [==============================] - 5s 17ms/step - loss: 0.0158 - mae: 0.1414 - val_loss: 0.0089 - val_mae: 0.1129
Epoch 2/40
93/93 [==============================] - 1s 9ms/step - loss: 0.0111 - mae: 0.1198 - val_loss: 0.0069 - val_mae: 0.0998
Epoch 3/40
93/93 [==============================] - 1s 9ms/step - loss: 0.0072 - mae: 0.0919 - val_loss: 0.0041 - val_mae: 0.0716
Epoch 4/40
93/93 [==============================] - 1s 9ms/step - loss: 0.0058 - mae: 0.0802 - val_loss: 0.0037 - val_mae: 0.0651
Epoch 5/40
93/93 [==============================] - 1s 9ms/step - loss: 0.0054 - mae: 0.0773 - val_loss: 0.0035 - val_mae: 0.0628
Epoch 6/40
93/93 [==============================] - 1s 9ms/step - loss: 0.0051 - mae: 0.0749 - val_loss: 0.0033 - val_mae: 0.0621
Epoch 7/40
93/93 [==============================] - 1s 9ms/step - loss: 0.0049 - mae: 0.0729 - val_loss: 0.0030 - val_mae: 0.0571
Epoch 8/40
93/93 [==============================] - 1s 9ms/step - loss: 0.0046 - mae: 0.0

In [224]:
total_mae = 0
i = 100
count = 0
for x, y in valid_set.take(i):
    y_pred = model(x)
    mae = np.mean(np.abs(y.numpy() - y_pred.numpy()))
    total_mae += mae
    count += 1
total_mae /= count
print(total_mae)

0.04675425466274648


# pytorch

In [225]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, series, window_size):
        self.series = series
        self.window_size = window_size

    def __len__(self):
        return len(self.series) - self.window_size

    def __getitem__(self, idx):
        X = self.series[idx : idx+self.window_size]
        y = self.series[idx+1 : idx+1+self.window_size]
        return X, y

window_size = 30
timeSeries_DS = TimeSeriesDataset(X_train, window_size)
x, y = timeSeries_DS[0]

In [226]:
x

array([0.24284279, 0.26192868, 0.29306881, 0.23304872, 0.35585133,
       0.34957308, 0.39678553, 0.27749874, 0.31767956, 0.31592165,
       0.66373682, 0.35660472, 0.30688096, 0.31767956, 0.37343044,
       0.36966349, 0.37669513, 0.41863385, 0.35735811, 0.43119036,
       0.38171773, 0.27498744, 0.26494224, 0.31567052, 0.29306881,
       0.18206931, 0.18960321, 0.23606228, 0.25414365, 0.21220492])

In [227]:
timeSeries_loader = DataLoader(timeSeries_DS, batch_size=32, shuffle=True)
x,y = next(iter(timeSeries_loader))
x.shape, y.shape

(torch.Size([32, 30]), torch.Size([32, 30]))

In [228]:
x[0,:10], y[0,:10]

(tensor([0.3330, 0.2722, 0.3546, 0.2941, 0.3380, 0.2504, 0.2069, 0.4915, 0.4448,
         0.3478], dtype=torch.float64),
 tensor([0.2722, 0.3546, 0.2941, 0.3380, 0.2504, 0.2069, 0.4915, 0.4448, 0.3478,
         0.4367], dtype=torch.float64))

In [229]:
np.where(sunspots == x[0,0].numpy())

(array([2617, 2749]),)

In [231]:
sunspots[2749:2749+window_size]

array([0.33299849, 0.27222501, 0.35459568, 0.29407333, 0.33802109,
       0.2503767 , 0.20693119, 0.49146158, 0.44475138, 0.34781517,
       0.43671522, 0.59241587, 0.48895028, 0.49045706, 0.36087393,
       0.47790055, 0.53164239, 0.5668006 , 0.50577599, 0.6702662 ,
       0.66197891, 0.65168257, 0.62682069, 0.56780512, 0.55097941,
       0.44876946, 0.58312406, 0.63962833, 0.5592667 , 0.48442993])

In [268]:
# causal padding을 지원하는 conv1d 층 구현
# 여기가 문제인 듯
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super().__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, **kwargs)
        self.kernel_size = kernel_size

    def forward(self, x):
        # kernel_size - 1 만큼 패딩을 오른쪽(미래 데이터 측)에 추가
        padding = (0,self.kernel_size - 1)
        # pad has the form (padding_left,padding_right)
        x = F.pad(x, pad=padding, mode='constant')  # 'constant'는 0으로 패딩한다는 의미
        return self.conv(x)

In [233]:
# 참고
F.pad(torch.Tensor([1,2,3,4]), pad=(3,0), mode='constant')

tensor([0., 0., 0., 1., 2., 3., 4.])

In [234]:
# 참고
print(x.shape)
print(x.unsqueeze(1).shape)
temp = CausalConv1d(in_channels=1, out_channels=60, kernel_size=5)
x_conv = temp(x.unsqueeze(1).float())
print(x_conv.shape)

torch.Size([32, 30])
torch.Size([32, 1, 30])
torch.Size([32, 60, 30])


In [235]:
# 참고
print(x_conv.permute(0,2,1).shape)
x_lstm1, _ = nn.LSTM(input_size=60, hidden_size=60, batch_first=True)(x_conv.permute(0,2,1))
print(x_lstm1.shape)
x_lstm2, _ = nn.LSTM(input_size=60, hidden_size=60, batch_first=True)(x_lstm1)
print(x_lstm2.shape)
x_dense1 = nn.Linear(in_features=60, out_features=30)(x_lstm2)
print(x_dense1.shape)

torch.Size([32, 30, 60])
torch.Size([32, 30, 60])
torch.Size([32, 30, 60])
torch.Size([32, 30, 30])


In [269]:
class timeSeriesModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1d = CausalConv1d(in_channels=1, out_channels=60, kernel_size=5)

        # Note: batch_first=True makes the input and output tensors of shape (batch, seq, feature)
        self.lstm1 = nn.LSTM(input_size=60, hidden_size=60, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=60, hidden_size=60, batch_first=True)
        self.dense1 = nn.Linear(in_features=60, out_features=30)
        self.dense2 = nn.Linear(in_features=30, out_features=10)
        self.dense3 = nn.Linear(in_features=10, out_features=1)

    def forward(self, x):
        if x.dim()==2:
            # channel
            x = x.unsqueeze(1)
        x = torch.relu(self.conv1d(x.float()))
        x = x.permute(0, 2, 1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = torch.relu(self.dense1(x))
        x = torch.relu(self.dense2(x))
        x = self.dense3(x)

        return x


In [270]:
timeSeries_model = timeSeriesModel()
x_final = timeSeries_model(x)
x_final.shape

torch.Size([32, 30, 1])

In [271]:
train_DS = TimeSeriesDataset(X_train, window_size)
train_loader = DataLoader(train_DS, batch_size=32, shuffle=True)
val_DS = TimeSeriesDataset(X_valid, window_size)
val_loader = DataLoader(val_DS, batch_size=32, shuffle=True)

In [272]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
timeSeries_model.to(device)
optimizer = torch.optim.SGD(timeSeries_model.parameters(), lr=1e-5, momentum=0.9)
criterion = nn.HuberLoss().to(device)

In [273]:
epochs = 40
for epoch in range(epochs):
    train_loss = 0
    train_mae = 0
    timeSeries_model.train()
    for inputs, targets in train_loader:
        inputs = inputs.float().to(device)
        targets = targets.float().to(device)
        outputs = timeSeries_model(inputs).squeeze()
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss +=  loss.item() * inputs.size(0)
        train_mae += torch.mean(torch.abs(targets - outputs)) * inputs.size(0)
    train_loss /= len(train_DS)
    train_mae /= len(train_DS)


    val_loss = 0
    val_mae = 0
    timeSeries_model.eval()
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.float().to(device)
            targets = targets.float().to(device)
            outputs = timeSeries_model(inputs).squeeze()
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
            val_mae += torch.mean(torch.abs(targets - outputs)) * inputs.size(0)
    val_loss /= len(val_DS)
    val_mae /= len(val_DS)

    print(f'''Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Train MAE: {train_mae:.6f},Val Loss: {val_loss:.6f}, Val MAE: {val_mae:.6f}''')

Epoch 1/40, Train Loss: 0.015806, Train MAE: 0.136988,Val Loss: 0.010059, Val MAE: 0.114591
Epoch 2/40, Train Loss: 0.015786, Train MAE: 0.136973,Val Loss: 0.010052, Val MAE: 0.114612
Epoch 3/40, Train Loss: 0.015767, Train MAE: 0.136961,Val Loss: 0.010046, Val MAE: 0.114637
Epoch 4/40, Train Loss: 0.015749, Train MAE: 0.136957,Val Loss: 0.010040, Val MAE: 0.114663
Epoch 5/40, Train Loss: 0.015731, Train MAE: 0.136946,Val Loss: 0.010034, Val MAE: 0.114687
Epoch 6/40, Train Loss: 0.015713, Train MAE: 0.136939,Val Loss: 0.010029, Val MAE: 0.114715
Epoch 7/40, Train Loss: 0.015695, Train MAE: 0.136929,Val Loss: 0.010024, Val MAE: 0.114747
Epoch 8/40, Train Loss: 0.015678, Train MAE: 0.136924,Val Loss: 0.010019, Val MAE: 0.114782
Epoch 9/40, Train Loss: 0.015662, Train MAE: 0.136920,Val Loss: 0.010014, Val MAE: 0.114818
Epoch 10/40, Train Loss: 0.015645, Train MAE: 0.136913,Val Loss: 0.010009, Val MAE: 0.114852
Epoch 11/40, Train Loss: 0.015629, Train MAE: 0.136909,Val Loss: 0.010005, Val 

In [267]:
total_mae = 0
counts = 0
for inputs, targets in val_loader:
    inputs = inputs.float().to(device)
    targets = targets.float().to(device)
    outputs = timeSeries_model(inputs).squeeze()
    mae = np.mean(np.abs(targets.cpu().detach().numpy() - outputs.detach().cpu().numpy()))
    total_mae += mae
    counts += 1
total_mae /= count
print(total_mae)

0.13647895199911936


In [ ]:
import zipfile

url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
urllib.request.urlretrieve(url, 'household_power.zip')
with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
    zip_ref.extractall()
hepc = pd.read_csv('household_power_consumption.csv')
hepc.head()